# 3D Object Detection Evaluation Tutorial

Welcome to the 3D object detection evaluation tutorial! We'll walk through the steps to submit your detections to the competition server.

In [1]:
from av2.evaluation.detection.eval import evaluate
from av2.evaluation.detection.utils import DetectionCfg
from pathlib import Path
from av2.utils.io import read_feather, read_all_annotations

### Constructing the evaluation configuration

The `DetectionCfg` class stores the configuration for the 3D object detection challenge.

- During evaluation, we remove _all_ cuboids which are not within the region-of-interest (ROI) which spatially is a 5 meter dilation of the drivable area isocontour. 

- **NOTE**: If you would like to _locally_ enable this behavior, you **must** pass in the directory to sensor dataset (to build the raster maps from the included vector maps).

In [2]:
dataset_dir = Path.home() / "data" / "datasets" / "av2" / "sensor"  # Path to your AV2 sensor dataset directory.
# dataset_dir = Path.home() / "data" / "argoverse2" / "train-000" / "sensor" / "train"  # Path to your AV2 sensor dataset directory.
competition_cfg = DetectionCfg(dataset_dir=dataset_dir)  # Defaults to competition parameters.

In [3]:
# split = "val"
split = "train"
gts = read_all_annotations(dataset_dir=dataset_dir, split=split)  # Contains all annotations in a particular split.
display(gts)

,timestamp_ns,track_uuid,category,length_m,width_m,height_m,qw,qx,qy,qz,tx_m,ty_m,tz_m,num_interior_pts
0,315969292759974000,0613bec9-112c-49c1-a489-703f3ac3e83a,BOLLARD,0.224374,0.184634,0.884048,-0.657798,0.0,0.0,0.753194,-2.997469,15.623137,0.041610,8
1,315969292759974000,37d3b0ba-b85d-4873-8c39-1cefcd527a0e,BOLLARD,0.208739,0.235460,0.703602,-0.649536,0.0,0.0,0.760331,-4.287559,15.765794,-0.022924,5
2,315969292759974000,ed600893-4f81-4de3-9648-2ae4584919f5,BOLLARD,0.375235,0.236669,1.368589,-0.703722,0.0,0.0,0.710476,56.410333,17.224839,0.707052,6
3,315969292759974000,dafa726f-e93d-4d8c-8587-b7de1ddae454,BOX_TRUCK,10.272509,3.160544,4.458092,0.971667,0.0,0.0,-0.236354,137.236284,-11.235186,6.566510,17
4,315969292759974000,ce1bc4c9-b628-4616-b7e2-7f1b46f3e2fb,BUS,11.905311,2.589931,3.000000,0.737617,0.0,0.0,-0.675219,16.907608,-40.938176,2.756544,514
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
585166,315967507659789000,c8ca67de-68f3-455d-b6b5-49f177074613,REGULAR_VEHICLE,4.030000,1.740000,1.480930,0.996689,0.0,0.0,-0.081307,43.391715,-8.125756,0.210195,17
585167,315967507659789000,cada3c1d-2d1b-4932-b0e0-95d23a3959f2,REGULAR_VEHICLE,5.475258,1.927687,1.900801,0.996449,0.0,0.0,-0.084201,-24.807791,7.734792,0.774175,347
585168,315967507659789000,e7023232-c9a5-4cfa-8e8e-01bff4f30420,REGULAR_VEHICLE,4.239266,2.283655,2.515700,0.996862,0.0,0.0,-0.079154,70.302228,-12.554422,0.616283,39
585169,315967507659789000,ed3c0e46-903c-462a-ace7-7e806fa109f8,REGULAR_VEHICLE,4.589813,2.030258,1.645100,0.098895,0.0,0.0,0.995098,29.028116,5.664312,-0.202507,216


## Preparing detections for submission.

The evaluation expects the following 14 fields within a `pandas.DataFrame`:

- `tx_m`: x-component of the object translation in the egovehicle reference frame.
- `ty_m`: y-component of the object translation in the egovehicle reference frame.
- `tz_m`: z-component of the object translation in the egovehicle reference frame.
- `length_m`: Object extent along the x-axis in meters.
- `width_m`: Object extent along the y-axis in meters.
- `height_m`: Object extent along the z-axis in meters.
- `qw`: Real quaternion coefficient.
- `qx`: First quaternion coefficient.
- `qy`: Second quaternion coefficient.
- `qz`: Third quaternion coefficient.
- `score`: Object confidence.
- `log_id`: Log id associated with the detection.
- `timestamp_ns`: Timestamp associated with the detection.
- `category`: Object category.

Additional details can be found in [SUBMISSION_FORMAT.md](../src/av2/evaluation/detection/SUBMISSION_FORMAT.md).

In [ ]:
# If you've already aggregated your detections into one file.
dts_path = Path("detections.feather")
dts = read_feather(dts_path)

dts, gts, metrics = evaluate(dts, gts, cfg=competition_cfg)  # Evaluate instances.

In [ ]:
display(metrics)

Finally, if you would like to submit to the evaluation server, you just need to export your detections into a `.feather` file. This can be done by:

```python
dts.to_feather("detections.feather")
```